In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from pickle import dump, load
import time
from sklearn.neighbors import BallTree


import itertools
from tqdm.auto import tqdm
tqdm.pandas()
import gc

In [2]:
start_time = time.time()

In [3]:
pairs = pd.read_pickle('../input/fourpoints-location-matching/train_pairs_raw.pkl')

In [4]:
pairs = pairs.sample(frac=0.5, random_state=1, ignore_index=True)
gc.collect()

69

In [5]:
pairs.shape

(8553728, 26)

In [6]:
pairs_sample = pd.read_csv('../input/foursquare-location-matching/pairs.csv').iloc[0:2,:]

In [7]:
# because my computer's RAM is not enough, replace column with more than 70% missing values to 0 and 1
del_list = ['url_1','url_2','phone_1','phone_2']
for col in tqdm(del_list):
    pairs[col] = pairs[col].notnull().astype('int')
    pairs_sample[col] = pairs_sample[col].notnull().astype('int')

  0%|          | 0/4 [00:00<?, ?it/s]

# code for pairs csv, but it's slow for 10,000,000+ rows of data

pairs = pd.read_csv('../input/fourpoints-location-matching/train_pairs_raw.csv',dtype={'id_1': 'object',
 'name_1': 'object',
 'latitude_1': 'float64',
 'longitude_1': 'float64',
 'address_1': 'object',
 'city_1': 'object',
 'state_1': 'object',
 'zip_1': 'object',
 'country_1': 'object',
 'url_1': 'object',
 'phone_1': 'object',
 'categories_1': 'object',
 'id_2': 'object',
 'name_2': 'object',
 'latitude_2': 'float64',
 'longitude_2': 'float64',
 'address_2': 'object',
 'city_2': 'object',
 'state_2': 'object',
 'zip_2': 'object',
 'country_2': 'object',
 'url_2': 'object',
 'phone_2': 'object',
 'categories_2': 'object',
 'match': 'bool'})

#pairs_load = pd.read_csv('../input/foursquare-location-matching/pairs.csv',chunksize=100)
#pairs = pd.concat(pairs_load, ignore_index=True)
#pairs = pairs.iloc[0:10000,:]
#del pairs_load # clear memory

In [8]:
dtype_dict = pairs_sample.dtypes.apply(lambda x: x.name).to_dict()
del pairs_sample

In [9]:
dtype_dict

{'id_1': 'object',
 'name_1': 'object',
 'latitude_1': 'float64',
 'longitude_1': 'float64',
 'address_1': 'object',
 'city_1': 'object',
 'state_1': 'object',
 'zip_1': 'object',
 'country_1': 'object',
 'url_1': 'int64',
 'phone_1': 'int64',
 'categories_1': 'object',
 'id_2': 'object',
 'name_2': 'object',
 'latitude_2': 'float64',
 'longitude_2': 'float64',
 'address_2': 'object',
 'city_2': 'object',
 'state_2': 'object',
 'zip_2': 'object',
 'country_2': 'object',
 'url_2': 'int64',
 'phone_2': 'int64',
 'categories_2': 'object',
 'match': 'bool'}

In [10]:
del pairs['index']
gc.collect()

89

In [11]:
pairs = pairs.astype(dtype_dict)

In [12]:
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8553728 entries, 0 to 8553727
Data columns (total 25 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_1          object 
 1   name_1        object 
 2   latitude_1    float64
 3   longitude_1   float64
 4   address_1     object 
 5   city_1        object 
 6   state_1       object 
 7   zip_1         object 
 8   country_1     object 
 9   url_1         int64  
 10  phone_1       int64  
 11  categories_1  object 
 12  id_2          object 
 13  name_2        object 
 14  latitude_2    float64
 15  longitude_2   float64
 16  address_2     object 
 17  city_2        object 
 18  state_2       object 
 19  zip_2         object 
 20  country_2     object 
 21  url_2         int64  
 22  phone_2       int64  
 23  categories_2  object 
 24  match         bool   
dtypes: bool(1), float64(4), int64(4), object(16)
memory usage: 1.5+ GB


In [13]:
# impute missing values
cat_col = pairs.select_dtypes(include = ['object']).columns
pairs[cat_col] = pairs[cat_col].fillna('')

Reduce memory

In [14]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in tqdm(df.columns):
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [15]:
pairs = reduce_mem_usage(pairs)

Memory usage of dataframe is 1574.39 MB


  0%|          | 0/25 [00:00<?, ?it/s]

Memory usage after optimization is: 787.52 MB
Decreased by 50.0%


In [16]:
train = pd.read_csv('../input/foursquare-location-matching/train.csv')
#train = reduce_mem_usage(train)

In [17]:
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8553728 entries, 0 to 8553727
Data columns (total 25 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id_1          category
 1   name_1        category
 2   latitude_1    float16 
 3   longitude_1   float16 
 4   address_1     category
 5   city_1        category
 6   state_1       category
 7   zip_1         category
 8   country_1     category
 9   url_1         int8    
 10  phone_1       int8    
 11  categories_1  category
 12  id_2          category
 13  name_2        category
 14  latitude_2    float16 
 15  longitude_2   float16 
 16  address_2     category
 17  city_2        category
 18  state_2       category
 19  zip_2         category
 20  country_2     category
 21  url_2         int8    
 22  phone_2       int8    
 23  categories_2  category
 24  match         float16 
dtypes: category(16), float16(5), int8(4)
memory usage: 787.5 MB


for col in tqdm(['latitude_1', 'longitude_1', 'latitude_2', 'longitude_2']):
    pairs[col] = pairs[col].astype('float16')

In [18]:
for col in tqdm(['name_1','address_1','name_2','address_2','categories_1','categories_2']):
    pairs[col] = pairs[col].astype('object')

  0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8553728 entries, 0 to 8553727
Data columns (total 25 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id_1          category
 1   name_1        object  
 2   latitude_1    float16 
 3   longitude_1   float16 
 4   address_1     object  
 5   city_1        category
 6   state_1       category
 7   zip_1         category
 8   country_1     category
 9   url_1         int8    
 10  phone_1       int8    
 11  categories_1  object  
 12  id_2          category
 13  name_2        object  
 14  latitude_2    float16 
 15  longitude_2   float16 
 16  address_2     object  
 17  city_2        category
 18  state_2       category
 19  zip_2         category
 20  country_2     category
 21  url_2         int8    
 22  phone_2       int8    
 23  categories_2  object  
 24  match         float16 
dtypes: category(10), float16(5), int8(4), object(6)
memory usage: 860.4+ MB


# generate dictionary above
pairs = pd.read_csv('../input/foursquare-location-matching/pairs.csv')
pairs.dtypes.apply(lambda x: x.name).to_dict()

Data preprocessing & Feature transformation:
1. location (latitude, longtitude): finding the distance between two variables
2. word preprocessing: remove url symbol, stop words removal

In [20]:
# 1. location
def distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = (np.sin(d_lat/2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(d_lon/2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

# remove url
def remove_URL(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub(r"http\S+", "", x))
    return df

# stop words removal

def list_to_string(lis):
    string = ''
    for i in tqdm(lis):
        string += i
        string += ' '
    return string[:-1]

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tqdm(tokens) if not i in stops]
    return result
    
def stop_remove(df,cols):
    stops = set(stopwords.words('english'))
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in stops]))
    return df

In [21]:
stop_columns = ['name_1','address_1','name_2','address_2']
url_columns = ['url_1','url_2']
pairs['distance'] = distance(pairs.latitude_1,pairs.longitude_1,pairs.latitude_2,pairs.longitude_2)

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
#pairs[url_columns] = remove_URL(pairs, url_columns)[url_columns]

In [23]:
pairs[stop_columns] = stop_remove(pairs, stop_columns)[stop_columns]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8553728 [00:00<?, ?it/s]

  0%|          | 0/8553728 [00:00<?, ?it/s]

  0%|          | 0/8553728 [00:00<?, ?it/s]

  0%|          | 0/8553728 [00:00<?, ?it/s]

text similarity: fuzzy, cosine similarity

In [24]:
def fuzzy_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_fuzzy"] = df.progress_apply(lambda x: lev.ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
    return df    

In [25]:
gc.collect()

123

In [ ]:
col_1 = ['name_1','address_1','categories_1']
col_2 = ['name_2','address_2','categories_2']
pairs = fuzzy_similarity(pairs, col_1, col_2)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8553728 [00:00<?, ?it/s]

In [ ]:
pairs.to_csv("train_pairs.csv", index=False)
pairs.to_pickle('./train_pairs.pkl')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

# download data
<a href="train_pairs.csv"> train_pairs </a>

<a href="./train_pairs.pkl"> train_pairs pickle </a>

# Appendix

read large data: https://www.kaggle.com/code/rohanrao/tutorial-on-reading-large-datasets/notebook

read data faster: https://towardsdatascience.com/%EF%B8%8F-load-the-same-csv-file-10x-times-faster-and-with-10x-less-memory-%EF%B8%8F-e93b485086c7